Channel PCA captures most significant patterns across frames for each channel, potentially highlighting the most prominent changes or features in the video content for that specific channel.
**Do we want that or are there better ways to capture spatial features per frame?**

In [1]:
!pip install scikit-learn tqdm --quiet

In [2]:
import os
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [3]:
def find_repo_root(path='.'):
    path = os.path.abspath(path)
    while not os.path.isdir(os.path.join(path, '.git')):
        parent = os.path.dirname(path)
        if parent == path:
            # We've reached the root of the file system without finding '.git'
            return None
        path = parent
    return path

repo_root = find_repo_root()
print("Repository Root:", repo_root)

Repository Root: None


In [4]:
def get_full_path(relative_path, repo_root):
    if not repo_root:
        raise ValueError("Repository root not found. Ensure you're inside a Git repository.")

    return os.path.join(repo_root, relative_path)


In [5]:
# step 1 function
def load_and_combine_tensors(stage_name, input_folder, num_videos):
    combined_tensor = []
    video_indices = {}

    for video_id in range(1, num_videos + 1):
        filename = f"{str(video_id).zfill(4)}_{stage_name}.pkl"
        file_path = os.path.join(input_folder, stage_name, filename)

        if os.path.exists(file_path):
            #print(f"Loading tensor from: {file_path}")
            with open(file_path, 'rb') as file:
                tensor = pickle.load(file)
                combined_tensor.append(tensor)
                # Track start and end indices for each video
                end_index = sum(t.shape[0] for t in combined_tensor)
                video_indices[str(video_id).zfill(4)] = (end_index - tensor.shape[0], end_index)

    if not combined_tensor:
        print("No tensors found to combine.")
        return None, None

    combined_tensor = np.concatenate(combined_tensor, axis=0)
    return combined_tensor, video_indices


In [6]:
def load_transformed_tensors(stage_name, transformation):
    # create path to FM folder
    folder_path = os.path.join("preprocessed_videos_30frames_transformed", transformation, stage_name)
    
    combined_tensor = []
    video_indices = {}
    video_id = 901  # Initialize video_id to start from 901

    # List all .pkl files in the directory
    pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
    
    for filename in pkl_files:
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'rb') as file:
            tensor = pickle.load(file)
            combined_tensor.append(tensor)
            # Track start and end indices for each video
            end_index = sum(t.shape[0] for t in combined_tensor)
            video_indices[str(video_id).zfill(4)] = (end_index - tensor.shape[0], end_index)
            video_id += 1  # Increment video_id for the next iteration

    if not combined_tensor:
        print("No tensors found to combine.")
        return None, None

    combined_tensor = np.concatenate(combined_tensor, axis=0)
    return combined_tensor, video_indices

In [7]:
# Example usage
input_folder = 'preprocessed_videos_30frames'
output_folder = os.getcwd()
stages = ["stage_5"] #["stage_1", "stage_2", "stage_3", "stage_4", "stage_5"] #sucess: "stage_1", failed: "stage_2" when scaling
stage_name = "stage_5"
variance_ratio = 5

# Use the current working directory or a known absolute path
current_working_directory = os.getcwd()
stage_folder = os.path.join(current_working_directory, input_folder, stage_name)
print("Attempting to access:", stage_folder)

if not os.path.exists(stage_folder):
    print("Directory not found:", stage_folder)

# Calculate the number of video files in the folder
num_videos = len([f for f in os.listdir(stage_folder) if os.path.isfile(os.path.join(stage_folder, f))])
print(f"Number of videos found: {num_videos}")

Attempting to access: /work/pca/preprocessed_videos_30frames/stage_5
Number of videos found: 1000


In [8]:
training_tensor, training_indices = load_and_combine_tensors(stage_name, input_folder, num_videos)
test_tensor, test_indices = load_transformed_tensors(stage_name, 'deformed')
print(training_tensor.shape, len(training_indices))
print(test_tensor.shape, len(test_indices))

(30000, 1, 7, 7, 2048) 1000
(3000, 7, 7, 2048) 100


In [9]:
print(training_indices)

{'0001': (0, 30), '0002': (30, 60), '0003': (60, 90), '0004': (90, 120), '0005': (120, 150), '0006': (150, 180), '0007': (180, 210), '0008': (210, 240), '0009': (240, 270), '0010': (270, 300), '0011': (300, 330), '0012': (330, 360), '0013': (360, 390), '0014': (390, 420), '0015': (420, 450), '0016': (450, 480), '0017': (480, 510), '0018': (510, 540), '0019': (540, 570), '0020': (570, 600), '0021': (600, 630), '0022': (630, 660), '0023': (660, 690), '0024': (690, 720), '0025': (720, 750), '0026': (750, 780), '0027': (780, 810), '0028': (810, 840), '0029': (840, 870), '0030': (870, 900), '0031': (900, 930), '0032': (930, 960), '0033': (960, 990), '0034': (990, 1020), '0035': (1020, 1050), '0036': (1050, 1080), '0037': (1080, 1110), '0038': (1110, 1140), '0039': (1140, 1170), '0040': (1170, 1200), '0041': (1200, 1230), '0042': (1230, 1260), '0043': (1260, 1290), '0044': (1290, 1320), '0045': (1320, 1350), '0046': (1350, 1380), '0047': (1380, 1410), '0048': (1410, 1440), '0049': (1440, 147

In [7]:
# Step 2: globalized standardization (only based on training set)
def standardize_tensors(combined_tensor, transformed_tensor, video_indices, training_end_id='0800'):
    # Reshape both tensors
    reshaped_training_tensor = combined_tensor.reshape(combined_tensor.shape[0], -1)
    reshaped_test_tensor = transformed_tensor.reshape(transformed_tensor.shape[0], -1)
   
    scaler = StandardScaler()

    # Find the end index of the training set
    training_end_index = video_indices[training_end_id][1]

    # Fit the scaler only on the training set
    scaler.fit(reshaped_training_tensor[:training_end_index])

    standardized_training_tensor = scaler.transform(reshaped_training_tensor[:training_end_index])
    standardized_training_tensor = standardized_training_tensor.reshape(np.squeeze(combined_tensor[:training_end_index], axis=1).shape)

    # Transform the test set using the scaler fitted on the training set
    standardized_test_tensor = scaler.transform(reshaped_test_tensor)
    # Reshape the transformed data back to the original shape
    standardized_test_tensor = standardized_test_tensor.reshape(transformed_tensor.shape)
    
    return standardized_training_tensor, standardized_test_tensor

In [22]:
standardized_training_tensor, standardized_test_tensor = standardize_tensors(training_tensor, test_tensor, training_indices)
print(standardized_training_tensor.shape)
print(standardized_test_tensor.shape)

(30000, 7, 7, 2048)
(30000, 100352)
(3000, 100352)
(24000, 100352)
(24000, 7, 7, 2048)
(3000, 7, 7, 2048)


In [ ]:
# # Step 3: Separate the standardized tensor back into individual tensors
# def separate_standardized_tensor(standardized_tensor, video_indices):
#     separated_tensors = {}
#     for video_id, (start, end) in video_indices.items():
#         separated_tensors[video_id] = standardized_tensor[start:end, :]
#     return separated_tensors

In [ ]:
# seperated_test_tensors = separate_standardized_tensor(standardized_test_data, test_indices)
# for key, value in seperated_test_tensors.items():
#     print(f'{key}: {value.shape}')

In [ ]:
# # to get same number of PCs for each video: fit PCA on all videos with a given variance threshold. Find max number of components. Fit and transform PCA with max number of components. -> makes sure that variance captured in each video is >= variance_ratio.
# 
# def apply_fm_pca_and_save(separated_tensors, stage_name, output_folder, variance_ratio):
#     # Assuming 'separated_tensors' is your dictionary with video IDs as keys and feature maps as values
#     
#     # Initialize a dictionary to store the final PCA results for each video
#     final_pca_results = {}
#     # Determine the spatial dimensions product from the first tensor
#     first_feature_map = next(iter(separated_tensors.values()))
#     spatial_dims_product = np.prod(first_feature_map.shape[2:4])  # Assuming spatial dimensions are in 3rd and 4th place
#     
#     # # Step 1: Determine the maximum number of components needed, but only for training set
#     # max_components = 0
#     # training_video_ids = [vid for vid in separated_tensors if vid <= "0800"]
#     # for video_id in tqdm(training_video_ids, desc="Finding max. number of PCs..."):
#     #     feature_maps = separated_tensors[video_id]
#     #     for channel in range(feature_maps.shape[-1]):
#     #         pca = PCA(n_components=variance_ratio)
#     #         data_for_channel = feature_maps[..., channel].reshape(-1, spatial_dims_product)
#     #         pca.fit(data_for_channel)
#     #         max_components = max(max_components, pca.n_components_)
#     max_components = variance_ratio
#     print(f"Max. number of PCs: {max_components}")
#     
#     # Step 2: Apply PCA with the determined number of components
#     for video_id, feature_maps in tqdm(separated_tensors.items(), desc="Performing PCA..."):
#         pca_results = []
#     
#         # Loop over each channel
#         for channel in range(feature_maps.shape[-1]):
#             # Reshape the data for this channel
#             data_for_pca = feature_maps[..., channel].reshape(-1, spatial_dims_product)
#     
#             # Apply PCA with the maximum number of components
#             pca = PCA(n_components=max_components)
#             pca_result = pca.fit_transform(data_for_pca)
#     
#             pca_results.append(pca_result)
#     
#         # Concatenate the PCA results from all channels for this video
#         final_result = np.concatenate(pca_results, axis=1)
#     
#         # Store the result in the dictionary with the video ID as the key
#         final_pca_results[video_id] = final_result
#         # final_pca_results now contains the PCA-transformed data for each video
#         # print(f"Processed Video ID: {video_id}, Resulting Shape: {final_result.shape}")
# 
#     pca_folder = os.path.join(output_folder, f"PCA_channel_{variance_ratio}", stage_name)
#     if not os.path.exists(pca_folder):
#         os.makedirs(pca_folder)
# 
#     stage_number = stage_name[-1]
#     file_path = os.path.join(pca_folder, f'stage_{stage_number}_pca.pkl')
#     with open(file_path, 'wb') as f:
#         pickle.dump(final_pca_results, f)
#     
#     print(f"{stage_name} PCs stored in: {file_path}")

In [8]:
def apply_filter_pca_and_save(standardized_training_tensor, standardized_test_tensor,  transformation, stage_name, output_folder, n_components, seed=42):
    # create filter batches for pca
    num_batches = 8
    batch_size = standardized_training_tensor.shape[-1] // num_batches
    print(batch_size)
    # Calculate the product of the spatial dimensions
    spatial_dims_prod = np.prod(standardized_training_tensor.shape[1:3])
    print(spatial_dims_prod)
    
    pca_results = []
    pca_models = []

    for i in range(num_batches):
        # Extract the batch from the training tensor
        training_batch = standardized_training_tensor[..., i*batch_size : (i+1)*batch_size]
        
        # Reshape the training batch for PCA: flattening the spatial dimensions and channels per 30 frames
        n_segments = training_batch.shape[0] // 30
        reshaped_training_batch = np.zeros((n_segments, spatial_dims_prod * batch_size * 30))
        for seg_i in range(n_segments):
            segment = training_batch[seg_i*30 : (seg_i+1)*30].reshape(-1)
            reshaped_training_batch[seg_i, :] = segment
        
        print(f"Shape of flattened training filter batch {i}: {reshaped_training_batch.shape}")

        # Apply PCA on the reshaped training batch
        pca = PCA(n_components=n_components, random_state=seed)
        pca.fit(reshaped_training_batch)
        pca_models.append(pca)

    # After training PCA on each batch, apply transformation to the test tensor
    for i, pca in enumerate(pca_models):
        # Extract the batch from the test tensor similar to the training tensor processing
        test_batch = standardized_test_tensor[..., i*batch_size : (i+1)*batch_size]
        
        # Reshape the test batch similar to the training batch
        reshaped_test_batch = np.zeros((test_batch.shape[0] // 30, spatial_dims_prod * batch_size * 30))
        for seg_i in range(reshaped_test_batch.shape[0]):
            segment = test_batch[seg_i*30 : (seg_i+1)*30].reshape(-1)
            reshaped_test_batch[seg_i, :] = segment

        print(f"Shape of flattened test filter batch {i}: {reshaped_test_batch.shape}")
        
        # Apply PCA transformation to the reshaped test batch
        pca_result = pca.transform(reshaped_test_batch)
        print(f"Number of PCs in test filter batch {i}: {pca_result.shape}")
        # Append the PCA result
        pca_results.append(pca_result)


    # for i in range(num_batches):
    #     # Extract the batch
    #     training_batch = standardized_training_tensor[..., i*batch_size : (i+1)*batch_size]
    #            
    #     # Reshape the batch for PCA: flattening the spatial dimensions and channels per 30 frames
    #     n_segments = training_batch.shape[0] // 30
    #     reshaped_training_batch = np.zeros((n_segments, spatial_dims_prod * batch_size * 30))
    #     for seg_i in range(n_segments):
    #         segment = training_batch[seg_i*30 : (seg_i+1)*30].reshape(-1)
    #         reshaped_training_batch[seg_i, :] = segment
    #     
    #     print(f"Shape of flattened training filter batch {i}: {reshaped_training_batch.shape}")
    # 
    #     # Apply PCA
    #     pca = PCA(n_components=n_components, random_state=seed)
    #     pca.fit(reshaped_batch)
    #     pca_result = pca.transform(standardized_test_tensor)
    #     print(f"Number of PCs in filter batch {i}: {pca_result.shape}")
    #     # Append the PCA result
    #     pca_results.append(pca_result)
    #     
    #     # print(len(pca_results))

    # Concatenate the PCA results from all slices
    final_pca_results = np.concatenate(pca_results, axis=1)
    print(final_pca_results.shape)
    # save PCA results
    pca_folder = os.path.join(output_folder, f"PCA_filter_transformed_{n_components}", transformation, stage_name)
    if not os.path.exists(pca_folder):
        os.makedirs(pca_folder)
    
    stage_number = stage_name[-1]
    file_path = os.path.join(pca_folder, f'stage_{stage_number}_pca.pkl')
    with open(file_path, 'wb') as f:
        pickle.dump(final_pca_results, f)
    
    print(f"{stage_name} PCs stored in: {file_path}")

In [9]:
def process_stage_for_pca(input_folder, output_folder, stage_name, transformation, variance_ratio):
    """
    Process all videos of a given stage: standardize, apply PCA, and save the PCA-transformed tensors.
    Args:
    - input_folder: Folder containing the pre-processed videos.
    - output_folder: Folder to save PCA results.
    - stage_name: Name of the stage to process.
    Returns:
    - DataFrame containing metadata (video ID and variance captured).
    """
     # Use the current working directory or a known absolute path
    current_working_directory = os.getcwd()
    stage_folder = os.path.join(current_working_directory, input_folder, stage_name)
    print("Attempting to access:", stage_folder)

    if not os.path.exists(stage_folder):
        print("Directory not found:", stage_folder)
        return None
    # Calculate the number of video files in the folder
    num_videos = len([f for f in os.listdir(stage_folder) if os.path.isfile(os.path.join(stage_folder, f))])
    print(f"Number of videos found: {num_videos}")

    # Step 1: Load and combine tensors
    training_tensor, training_indices = load_and_combine_tensors(stage_name, input_folder, num_videos)
    test_tensor, test_indices = load_transformed_tensors(stage_name, transformation)
    print(training_tensor.shape, len(training_indices))
    print(test_tensor.shape, len(test_indices))
    print("Step 1 done.")
    
    # Step 2: Globally standardize the tensor
    standardized_training_tensor, standardized_test_tensor = standardize_tensors(training_tensor, test_tensor, training_indices)
    print(standardized_training_tensor.shape)
    print(standardized_test_tensor.shape)
    
    # # save standardized tensors
    # st_folder = os.path.join(output_folder, f"fm_standardized", stage_name)
    # if not os.path.exists(st_folder):
    #     os.makedirs(st_folder)
    # file_path = os.path.join(st_folder, f'fm_standardized_{stage_name}.pkl')
    # with open(file_path, 'wb') as f:
    #     pickle.dump(standardized_tensor, f)
    # print(f"{stage_name} standardized tensors stored in: {file_path}")
    
    print("Step 2 done.")

    # Step 3: Apply PCA to each tensor and save the result
    n_components = variance_ratio
    apply_filter_pca_and_save(standardized_training_tensor, standardized_test_tensor, transformation, stage_name, output_folder, n_components)
    print("Step 3 done.")

In [10]:
print(os.getcwd())
print(repo_root)

/work/pca
None


In [11]:
# Example usage
input_folder = 'preprocessed_videos_30frames'
output_folder = os.getcwd()
stages = ["stage_4", "stage_5"] #["stage_1", "stage_2", "stage_3", "stage_4", "stage_5"] #sucess: "stage_1", failed: "stage_2" when scaling
transformations = ['deformation', 'flipping', 'rotation']
variance_ratio = 0.95

# Iterate over each stage and process it
for transformation in transformations:
    for stage in stages:
        print(f"Processing {transformation} {stage}...")
        process_stage_for_pca(input_folder, output_folder, stage, transformation, variance_ratio)

Processing deformation stage_4...
Attempting to access: /work/pca/preprocessed_videos_30frames/stage_4
Number of videos found: 1000
(30000, 1, 14, 14, 1024) 1000
(3000, 14, 14, 1024) 100
Step 1 done.
(24000, 14, 14, 1024)
(3000, 14, 14, 1024)
Step 2 done.
128
196
Shape of flattened training filter batch 0: (800, 752640)
Shape of flattened training filter batch 1: (800, 752640)
Shape of flattened training filter batch 2: (800, 752640)
Shape of flattened training filter batch 3: (800, 752640)
Shape of flattened training filter batch 4: (800, 752640)
Shape of flattened training filter batch 5: (800, 752640)
Shape of flattened training filter batch 6: (800, 752640)
Shape of flattened training filter batch 7: (800, 752640)
Shape of flattened test filter batch 0: (100, 752640)
Number of PCs in test filter batch 0: (100, 718)
Shape of flattened test filter batch 1: (100, 752640)
Number of PCs in test filter batch 1: (100, 720)
Shape of flattened test filter batch 2: (100, 752640)
Number of PC

In [ ]:
#-----------------------------------------------------------------------

In [ ]:
# zip pca folder
import shutil

directory_to_zip = "PCA_channel_0.95"  # Replace with your directory name
output_filename = "PCA_channel_0.95"  # Replace with your desired output name
output_path = os.path.join(os.getcwd(), output_filename)
shutil.make_archive(output_path, 'zip', directory_to_zip)